In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .appName("HipsterIndex") \
    .config("spark.driver.extraClassPath", "/Users/szu004/miniconda2/envs/hail/lib/python3.6/site-packages/hail/hail-all-spark.jar")\
    .config("spark.jars", "/Users/szu004/miniconda2/envs/hail/lib/python3.6/site-packages/hail/hail-all-spark.jar,../target/variant-spark_2.11-0.3.0-SNAPSHOT-all.jar")\
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryo.registrator", "is.hail.kryo.HailKryoRegistrator") \
    .getOrCreate()

In [2]:
import hail as hl
hl.init(sc=spark.sparkContext)

using hail jar at /Users/szu004/miniconda2/envs/hail/lib/python3.6/site-packages/hail/hail-all-spark.jar
Running on Apache Spark version 2.4.1
SparkUI available at http://140.253.176.47:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.16-6da0d3571629
LOGGING: writing to /Users/szu004/dev/variant-spark/dev-notebooks/hail-20190716-1542-0.2.16-6da0d3571629.log


In [3]:
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

Loading BokehJS ...

In [4]:
data = hl.import_vcf('../data/hipsterIndex/hipster.vcf.bgz')

In [5]:
labels = hl.import_table('../data/hipsterIndex/hipster_labels.txt', delimiter=',', 
                types=dict(label='float64', score='float64')).key_by('samples')

2019-07-16 15:42:55 Hail: INFO: Reading table with no type imputation
  Loading column 'samples' as type 'str' (type not specified)
  Loading column 'score' as type 'float64' (user-specified)
  Loading column 'label' as type 'float64' (user-specified)



In [9]:
mt = data.annotate_cols(hipster = labels[data.s])
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'hipster': struct {
        score: float64, 
        label: float64
    }
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AA: str, 
        AC: array<int32>, 
        AF: array<float64>, 
        AFR_AF: array<float64>, 
        AMR_AF: array<float64>, 
        AN: int32, 
        CIEND: array<int32>, 
        CIPOS: array<int32>, 
        CS: str, 
        DP: int32, 
        EAS_AF: array<float64>, 
        END: int32, 
        EUR_AF: array<float64>, 
        EX_TARGET: bool, 
        IMPRECISE: bool, 
        MC: array<str>, 
        MEINFO: array<str>, 
        MEND: int32, 
        MLEN: int32, 
        MSTART: int32, 
        MULTI_ALLELIC: bool, 
        NS: int32, 
        SAS_AF: array<float64>, 

In [10]:
mt.count()

(17010, 2504)

In [23]:
from typing import *
from hail.expr.expressions import *
from hail.expr.types import *
from hail.typecheck import *
from hail.ir import *
from hail.table import Table


class RandomForestModel(object):
    
    def __init__(self,_mir):
        self._mir = _mir
        self._jrf_model = _jrf_model = Env.jvm().au.csiro.variantspark.hail.methods.RFModel.pyApply(Env.spark_backend('rf')._to_java_ir(self._mir))
    
    def fit_trees(self, n_trees = 500, batch_size = 100):
        self._jrf_model.fitTrees(n_trees, batch_size)
    
    def oob_error(self):
        return self._jrf_model.oobError()
    
    def variable_importance(self):
        return Table._from_java(self._jrf_model.variableImportance())                      
                                 
        

@typecheck(y=expr_float64,
           x=expr_int32,
           covariates=sequenceof(expr_float64))
def random_forest_model(y, x, covariates=()):
    
    mt = matrix_table_source('random_forest_model/x', x)
    check_entry_indexed('random_forest_model/x', x)
    mts = mt._select_all(col_exprs=dict(y=y),
                        row_exprs=dict(),
                        col_key=[],
                        entry_exprs=dict(e=x))
    return RandomForestModel(mts._mir)



In [20]:
rf_model = random_forest_model(y=mt.hipster.label,
                    x=mt.GT.n_alt_alleles())
rf_model.fit_trees()

In [26]:
print(rf_model.oob_error())
impTable = rf_model.variable_importance()
impTable.show(5)

0.18370607028753994


2019-07-16 15:57:12 Hail: INFO: Coerced sorted dataset


locus,alleles,importance
locus<GRCh37>,array<str>,float64
2:109511398,"[""G"",""A""]",7.54e-03
2:109511454,"[""C"",""A""]",2.67e-03
2:109511463,"[""G"",""A""]",1.29e-01
2:109511467,"[""GACTC"",""G""]",3.82e-01
2:109511478,"[""C"",""T""]",0.00e+00


In [32]:
gwas = hl.linear_regression_rows(y=mt.hipster.score,
                                 x=mt.GT.n_alt_alleles(),
                                 covariates=[1.0],
                                 pass_through=[mt.rsid])

2019-07-16 15:58:16 Hail: INFO: linear_regression_rows: running on 2504 samples for 1 response variable y,
    with input variable x, and 1 additional covariate...


In [29]:
gwas.show(5)

locus,alleles,rsid,n,sum_x,y_transpose_x,beta,standard_error,t_stat,p_value
locus<GRCh37>,array<str>,str,int32,float64,float64,float64,float64,float64,float64
2:109511398,"[""G"",""A""]","""rs150055772""",2504,4.00e+00,3.50e+01,-7.03e-01,1.62e+00,-4.34e-01,6.64e-01
2:109511454,"[""C"",""A""]","""rs558429529""",2504,1.00e+00,1.15e+01,2.05e+00,3.24e+00,6.33e-01,5.27e-01
2:109511463,"[""G"",""A""]","""rs200762071""",2504,1.47e+02,1.30e+03,-5.79e-01,2.60e-01,-2.23e+00,2.61e-02
2:109511467,"[""GACTC"",""G""]","""rs145115545""",2504,5.47e+02,5.06e+03,-2.07e-01,1.42e-01,-1.45e+00,1.47e-01
2:109511478,"[""C"",""T""]","""rs540842456""",2504,1.00e+00,1.20e+01,2.55e+00,3.24e+00,7.88e-01,4.31e-01


In [33]:
gwas_with_imp = gwas.join(impTable)

In [35]:
p = hl.plot.scatter(-hail.log10(gwas_with_imp.p_value),
                    gwas_with_imp.importance, 
                    hover_fields=dict(rs=gwas_with_imp.rsid, loc=gwas_with_imp.locus))
show(p)

In [36]:
p = hl.plot.manhattan(gwas_with_imp.p_value, hover_fields=dict(ri=gwas_with_imp.rsid))
show(p)